# Acceso a los datos de la web

### Web Scraping

El __Web Scraping__ (o Scraping) son un conjunto de técnicas que se utilizan para obtener de forma automática el contenido que hay en páginas web a través de su código HTML. 
* Es una opción cuando no hay API's para extraer datos de la web
* Otros términos:
    * "Spydering the web"
    * "Web crawling"

Las técnicas de Scraping se pueden enmarcar dentro del campo del Big Data en la primera fase de recolección de datos para su posterior almacenamiento, tratamiento y visualización.

El uso de estas técnicas tienen como finalidad recopilar grandes cantidades de datos de diferentes páginas web cuyo uso posterior puede ser muy variado:

  * homogenización de datos, 
  * tratamiento de contenido para la extracción de conocimiento, 
  * complementar datos en una web, etc.


__Realizar una petición__

La página web con la que vamos a jugar es:
"https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid"
 que contiene datos de la lista de municipios de la Comunidad de Madrid.
 
Para acceder al contenido de una página web usamos el protocolo HTTP Request/Response. 

A continuación adjunto una función capaz de hacer una conexión HTTP para acceder a una página web y extraer información.



In [1]:
import requests

url = "https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid"

# Realizamos la petición HTTP a la web
response = requests.get(url)

* La función __get__ del módulo __request__ de Python abre una conexión con el servidor donde se encuentra la url (parámetro de  __get__) y manda la petición.

* La respuesta del servidor se almacena en la variable (u objeto) _response_.

* A partir del objeto _response_, que almacena muchos datos relacionados con la petición HTTP (cabeceras, cookies, etc.) obtenemos el __status_code__ y el __HTML__ (como un string) de la web. 

In [2]:
response.status_code        # atributo status_code: nos indicará si la petición 
                            # ha tenido éxito

200

In [3]:
# Comprobamos que la petición nos devuelve un Status Code = 200
statusCode = response.status_code 

if statusCode == 200:
    print('La petición ha ido bien')
else:
    print('Problemas con la petición...')

La petición ha ido bien


A partir del objeto __response__, que almacena muchos datos relacionados con la petición HTTP (cabeceras, cookies, etc.) obtenemos el __status_Code__ y el __HTML__ (como un string) de la web. 

In [4]:
htmlText = response.text


__Extraer información__

A partir del string __htmlText__ que representa el código interno de la pagina web, parseamos la web para extraer la información que nos interesa.

Mecanismos:

* Usar expresiones regulares para localizar cadenas: esto puede ser muy laborioso, y pesado ...
    
    
    
* Usar alguna de las librerías de Python que permiten parsear el texto HTML de forma cómoda. 
    * __BeautifullSoup__ 
    * __Scrapy__
    
    
__BeautifulSoup__  nos aporta los métodos necesarios (y muy bien optimizados) para obtener el contenido que hay entre las etiquetas HTML.

In [5]:
from bs4 import BeautifulSoup

# Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
soup = BeautifulSoup(htmlText, 'html.parser')


### Recuperar los datos de la tabla

Las tablas de HTML se definen con tres etiquetas: `<table>` para crear la tabla, `<tr>` para crear cada fila y `<td>` para crear cada columna.

In [6]:
from IPython.display import HTML
s = """
<table>
    <tr>
      <th><strong>Curso</strong></th>
      <th><strong>Horas</strong></th>
    </tr>
 
    <tr>
      <td>CSS</td>
      <td>20</td>
    </tr>
 
    <tr>
      <td>HTML</td>
      <td>20</td>
    </tr>
</table>"""
HTML(s)

Curso,Horas
CSS,20
HTML,20


__Ejemplo:__ Queremos recuperar los datos de la tabla que aparece en la página  [Municipios Madrid](https://es.wikipedia.org/wiki/Anexo:Municipios_de_la_Comunidad_de_Madrid).

En este caso utilizamos el método __find_all()__ que lo que hace es coger todos los fragmentos del HTML que correpondan a las etiquetas __tr__ y __td__.

In [7]:
content = {}
# Obtenemos cada una de las filas de la tabla 
rows = soup.find_all('tr')
for r in rows:
    #Seleccionamos las celdas de la tabla (td)
    celdas=r.find_all('td')
    # ignoramos la primera celda, que no tiene elementos td sino th (ver HTML de la página web)
    if len(celdas)>0:
        # En lugar de un separador de miles, se ha usado un caracter parecido a un espacio en blanco
        # por lo que en la celda de habitantes hay que eliminar todos los caracteres que no sean números
        #content.append([celdas[0].string, ''.join(c for c in celdas[1].string if c.isdigit())])
        content[celdas[0].string] = { 'población': int( ''.join(c for c in celdas[1].string if c.isdigit())),
                                      'superficie': float(celdas[2].string.replace(',', '.') ),
                                      'altitud': int( ''.join(c for c in celdas[6].string if c.isdigit())),}


In [8]:
content['Valdemorillo']

{'altitud': 818, 'población': 12168, 'superficie': 93.68}

#### 1 Ejercicio

Calcula la población total de todos los municipios.

In [136]:
sum ([i['población'] for i in content.values()])

6466996

#### 2 Ejercicio

Obtener un listado de los municipios cuya altitud sea por encima de los 700 metros.

In [158]:
# Sol:
[i[0]  for i in  zip(content.keys(),content.values()) if (i[1]['altitud']>700)]

['La Acebeda',
 'Alameda del Valle',
 'Alcorcón',
 'Algete',
 'Alpedrete',
 'Anchuelo',
 'El Atazar',
 'Becerril de la Sierra',
 'Belmonte de Tajo',
 'El Berrueco',
 'Berzosa del Lozoya',
 'El Boalo',
 'Braojos de la Sierra',
 'Brea de Tajo',
 'Buitrago del Lozoya',
 'Bustarviejo',
 'Cabanillas de la Sierra',
 'La Cabrera',
 'Cadalso de los Vidrios',
 'Campo Real',
 'Canencia',
 'Cenicientos',
 'Cercedilla',
 'Cervera de Buitrago',
 'Chinchón',
 'Collado Mediano',
 'Collado Villalba',
 'Colmenar de Oreja',
 'Colmenar Viejo',
 'Colmenarejo',
 'Corpa',
 'El Escorial',
 'Fresnedillas de la Oliva',
 'Galapagar',
 'Garganta de los Montes',
 'Gargantilla del Lozoya y Pinilla de Buitrago',
 'Gascones',
 'Guadalix de la Sierra',
 'Guadarrama',
 'La Hiruela',
 'Horcajo de la Sierra-Aoslos',
 'Horcajuelo de la Sierra',
 'Hoyo de Manzanares',
 'Lozoya',
 'Lozoyuela-Navas-Sieteiglesias',
 'Madarcos',
 'Majadahonda',
 'Manzanares el Real',
 'Miraflores de la Sierra',
 'El Molar',
 'Los Molinos',
 '